In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import segyio
import glob
import pandas as pd

import scipy as sp

from scipy.sparse.linalg import LinearOperator, cg, lsqr

import pylops
from pylops.utils                      import dottest
from pylops.utils.wavelets             import *
from pylops.optimization.sparsity  import *

# EXPERIMENT PARAMS
from pyfrac.hydrafrac.acquisition import recgeom,rec2das_datatrnsfrm,dasgeom

# LOCATION
from pyfrac.modelling.kirchhoff import Kirchhoff
from pyfrac.locationsolvers.imaging import *
from pyfrac.locationsolvers.crosscorr_imaging import xcorr_imaging


# VISUALISATION
from pyfrac.visualisation.traceviz import traceimage, wiggleplot, multiwiggleplot
from pyfrac.visualisation.eventimages import locimage3d
from pyfrac.visualisation.plotting_support import explode_volume


%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
local_modpath = '/home/birniece/Documents/data/ANPERC_Lab/STYLOLITES/StyloliteModels'

seispath = '/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/INCOMING/Stylolites/outputs/su'
modpath ='/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/Stylolites/inputs/StyloliteModels/'
figpath = './Figures/StyloliteWaveforms/'

# Load Modelled Data

In [4]:
def read_seis(seisfile, nr):
    # s_wave_file = os.path.join(data_path,'test.bin.curl')
    seis_flat = np.loadtxt(seisfile)

    # determine number of timesamples and ensure it saved fully
    nt = int(seis_flat.shape[0] / nr)
    try:
        seis_flat.reshape((nr, nt))
    except:
        print('Not good dims \n nt:%i nr:%i \n vz flat:%i' % (nt, nr, seis_flat.shape[0]))

    # Resample to 2D
    seis = seis_flat[:nt * nr].reshape((nr, nt))
    return seis

In [5]:
def get_FB_Coda_Energy(data, nr, fblim=1e-4, wavelength = 75):
    # Find First Breaks
    
    fb_locs = np.zeros([nr])
    fb_energy = np.zeros([nr])
    coda_energy = np.zeros([nr])
    
    for tr in range(nr):
        fb_locs[tr] = [n for n,i in enumerate(data[tr]) if abs(i)>fblim][0]
        
    for tr in range(nr):    
        fb_energy[tr] = np.sum(data[tr,int(fb_locs[tr]):int(fb_locs[tr])+wavelength]**2)
        coda_energy[tr] = np.sum(data[tr,int(fb_locs[tr])+wavelength:]**2)
    
    return fb_locs, fb_energy, coda_energy

In [10]:
vz_file_list = glob.glob(seispath+'/*div.txt')

fb_energy = np.zeros([len(vz_file_list)])
coda_energy = np.zeros([len(vz_file_list)])
expname_list = []

for i,vz_file in enumerate(vz_file_list):
    expname = vz_file.split('/')[-1][:-8]
    expname_list.append(expname)
    print(expname)
    
    vz = read_seis(vz_file, nr=150)
    _, fb_energy_tr, coda_energy_tr = get_FB_Coda_Energy(vz, nr=150, fblim=1e-2, wavelength = 75)
    fb_energy[i] = np.mean(fb_energy_tr)
    coda_energy[i] = np.mean(coda_energy_tr)

InclinedStylo_a-15_pc70
StraightStylo_w02_pc90
InclinedStylo_a45_pc80
InclinedStylo_a15_pc90
PinchOutStylo_pc70
InclinedStylo_a-05_pc90
SineStylo_h20_pc90
InclinedStylo_a05_pc90
SineStylo_h15_pc90
InclinedStylo_a-35_pc80
InclinedStylo_a-15_pc90
InclinedStylo_a25_pc70
SineStylo_h05_pc80
InclinedStylo_a-25_pc80
InclinedStylo_a05_pc70
InclinedStylo_a05_pc80
InclinedStylo_a35_pc70
SineStylo_h20_pc80
InclinedStylo_a-45_pc80
StraightStylo_w03_pc90
StraightStylo_w04_pc80
StraightStylo_w05_pc90
StraightStylo_w03_pc80
InclinedStylo_a35_pc80
InclinedStylo_a-05_pc70
StraightStylo_w01_pc70
InclinedStylo_a-25_pc90
InclinedStylo_a45_pc90
StraightStylo_w05_pc70
InclinedStylo_a15_pc80
SineStylo_h10_pc90
InclinedStylo_a15_pc70
InclinedStylo_a-15_pc80
StraightStylo_w04_pc70
StraightStylo_w01_pc80
StraightStylo_w02_pc80
StraightStylo_w02_pc70
InclinedStylo_a-35_pc90
SineStylo_h05_pc70
PinchOutStylo_pc80
SineStylo_h20_pc70
StraightStylo_w03_pc70
InclinedStylo_a-25_pc70
InclinedStylo_a25_pc90
StraightStylo

In [11]:
df = pd.DataFrame({'experiment':expname_list, 
                   'fb_energy':fb_energy, 
                   'coda_energy':coda_energy, 
                  })
df.to_csv('./StylolitesWaveformEnergyAnalysis.csv')
df.head()

,experiment,fb_energy,coda_energy
0,InclinedStylo_a-15_pc70,9.950357e+08,2.298715e+06
1,StraightStylo_w02_pc90,1.005753e+09,1.241478e+06
2,InclinedStylo_a45_pc80,9.895033e+08,1.249638e+06
3,InclinedStylo_a15_pc90,9.990947e+08,1.186559e+06
4,PinchOutStylo_pc70,9.915319e+08,4.997630e+06


In [15]:
df.sort_values('fb_energy').tail(20)

,experiment,fb_energy,coda_energy
38,SineStylo_h05_pc70,1.001903e+09,2.078030e+06
26,InclinedStylo_a-25_pc90,1.002096e+09,1.200174e+06
34,StraightStylo_w01_pc80,1.002923e+09,1.325060e+06
12,SineStylo_h05_pc80,1.003291e+09,1.351985e+06
1,StraightStylo_w02_pc90,1.005753e+09,1.241478e+06
45,InclinedStylo_a35_pc90,1.005956e+09,1.169077e+06
35,StraightStylo_w02_pc80,1.006965e+09,2.228028e+06
13,InclinedStylo_a-25_pc80,1.007847e+09,1.517735e+06
22,StraightStylo_w03_pc80,1.009678e+09,3.833100e+06
19,StraightStylo_w03_pc90,1.009808e+09,1.475270e+06


In [ ]:
homo_div = read_seis(os.path.join(seispath,'Homogeneous_div.txt'), nr=150)
fb_locs, fb_energy, coda_energy = get_FB_Coda_Energy(homo_div, nr=150, fblim=1e-2, wavelength = 75)

fig, ax = plt.subplots(1,1,figsize=[15,3])
ax.plot(homo_div[0])
ax.axvline(fb_locs[0])
ax.axvline(fb_locs[0]+75)

In [9]:
fb_energy

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])